# Metric Learning Exercise

In [1]:
import torch
import os
import sys
from mmfashion.datasets import InShopDataset
from mmfashion.datasets.loader import build_dataloader
working_dir = os.getcwd()
sys.path.append(working_dir)
sys.path.append(os.path.join(working_dir, 'siamese-triplet'))

# Prepare Dataset

Working with the In-Shop dataset from mmfashion, please check https://github.com/open-mmlab/mmfashion/blob/master/docs/dataset/IN_SHOP_DATASET.md

In [2]:
img_path = os.path.join(working_dir, 'data/In-shop')
annotations_path = os.path.join(img_path, 'Anno')

In [3]:
batch_size = 256

In [4]:
class_mapping = {
    'Denim': 0,
    'Jackets_Vests': 1,
    'Pants': 2,
    'Shirts_Polos': 3,
    'Shorts': 4,
    'Suiting': 5,
    'Sweaters': 6,
    'Sweatshirts_Hoodies': 7,
    'Tees_Tanks': 8,
    'Blouses_Shirts': 9,
    'Cardigans': 10,
    'Dresses': 11,
    'Graphic_Tees': 12,
    'Jackets_Coats': 13,
    'Leggings': 14,
    'Rompers_Jumpsuits': 15,
    'Skirts': 16
}

train_dataset_siamese = InShopDataset(img_path=img_path,
                              img_file=os.path.join(annotations_path,'train_img.txt'),
                              label_file=os.path.join(annotations_path,'train_labels.txt'),
                              id_file=os.path.join(annotations_path,'train_id.txt'),
                              bbox_file=os.path.join(annotations_path,'train_bbox.txt'),
                              landmark_file=os.path.join(annotations_path,'train_landmarks.txt'),
                              img_size=[256, 256],
                              class_mapping=class_mapping)

query_dataset_siamese = InShopDataset(img_path=img_path,
                              img_file=os.path.join(annotations_path,'query_img.txt'),
                              label_file=os.path.join(annotations_path,'query_labels.txt'),
                              id_file=os.path.join(annotations_path,'query_id.txt'),
                              bbox_file=os.path.join(annotations_path,'query_bbox.txt'),
                              landmark_file=os.path.join(annotations_path,'query_landmarks.txt'),
                              img_size=[256, 256],
                              class_mapping=class_mapping)

cuda = torch.cuda.is_available()

#train_dataloader_siamese = torch.utils.data.DataLoader(train_dataset_siamese, batch_size=batch_size, shuffle=True, **kwargs)
#query_dataloader_siamese = torch.utils.data.DataLoader(query_dataset_siamese, batch_size=batch_size, shuffle=True, **kwargs)

train_dataloader_siamese = build_dataloader(dataset=train_dataset_siamese,
                                           imgs_per_gpu=batch_size,
                                           workers_per_gpu=1,
                                           num_gpus=1,
                                           shuffle=True)

query_dataloader_siamese = build_dataloader(dataset=query_dataset_siamese,
                                           imgs_per_gpu=batch_size,
                                           workers_per_gpu=1,
                                           num_gpus=1,
                                           shuffle=False)

n_classes = 17

# Utils to extract and plot embeddings in low dimensional space

In [5]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def plot_embeddings(embeddings, targets, labels, xlim=None, ylim=None):
    plt.figure(figsize=(10,10))
    for i in range(10):
        inds = np.where(targets==i)[0]
        plt.scatter(embeddings[inds,0], embeddings[inds,1], alpha=0.5, color=colors[i])
    if xlim:
        plt.xlim(xlim[0], xlim[1])
    if ylim:
        plt.ylim(ylim[0], ylim[1])
    plt.legend(labels)

def extract_embeddings(dataloader, model):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataloader.dataset), 2))
        labels = np.zeros(len(dataloader.dataset))
        k = 0
        for images, target in dataloader:
            if cuda:
                images = images.cuda()
            embeddings[k:k+len(images)] = model.get_embedding(images).data.cpu().numpy()
            labels[k:k+len(images)] = target.numpy()
            k += len(images)
    return embeddings, labels

In [6]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

import numpy as np
cuda = torch.cuda.is_available()
from networks import EmbeddingNet, ClassificationNet
from metrics import AccumulatedAccuracyMetric

embedding_net = EmbeddingNet()
model = ClassificationNet(embedding_net, n_classes=n_classes)
if cuda:
    model.cuda()
loss_fn = torch.nn.NLLLoss()
lr = 1e-2
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 50

In [7]:
from trainer import fit
fit(train_dataloader_siamese, query_dataloader_siamese, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AccumulatedAccuracyMetric()])

/usr/local/lib/python3.6/dist-packages/torch-1.4.0-py3.6-linux-x86_64.egg/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


RuntimeError: CUDA out of memory. Tried to allocate 1.94 GiB (GPU 0; 3.95 GiB total capacity; 2.13 GiB already allocated; 491.06 MiB free; 2.13 GiB reserved in total by PyTorch)

In [14]:
import torch
from torchvision.datasets import FashionMNIST
from torchvision import transforms

mean, std = 0.28604059698879553, 0.35302424451492237
batch_size = 256

train_dataset = FashionMNIST('../data/FashionMNIST', train=True, download=True,
                             transform=transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((mean,), (std,))
                             ]))
test_dataset = FashionMNIST('../data/FashionMNIST', train=False, download=True,
                            transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((mean,), (std,))
                            ]))

cuda = torch.cuda.is_available()
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

for loader in enumerate(train_loader):
    print (loader[0])
    print (loader[1][0])
    print (loader[1][1])

0
tensor([[[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8

4
tensor([[[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8

tensor([[[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.810

13
tensor([[[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.

tensor([8, 0, 6, 6, 2, 3, 1, 9, 3, 2, 3, 7, 8, 6, 0, 8, 8, 8, 9, 6, 3, 3, 8, 3,
        8, 1, 5, 4, 9, 1, 3, 2, 1, 3, 1, 2, 7, 7, 4, 0, 7, 2, 8, 7, 7, 6, 0, 2,
        5, 6, 9, 6, 4, 5, 5, 6, 5, 3, 5, 4, 9, 7, 0, 4, 5, 7, 6, 9, 8, 2, 9, 1,
        7, 0, 5, 4, 3, 3, 2, 4, 8, 1, 5, 8, 7, 4, 4, 0, 3, 8, 0, 7, 7, 7, 4, 9,
        0, 8, 0, 9, 8, 8, 8, 7, 9, 4, 2, 8, 6, 3, 7, 6, 7, 1, 9, 9, 6, 4, 1, 5,
        6, 4, 5, 2, 9, 8, 3, 2, 3, 5, 9, 5, 9, 0, 1, 1, 5, 5, 9, 4, 3, 3, 5, 1,
        7, 6, 2, 9, 0, 1, 6, 6, 0, 1, 6, 3, 1, 2, 9, 1, 9, 1, 0, 7, 3, 6, 7, 5,
        5, 5, 9, 5, 8, 2, 0, 2, 9, 5, 8, 7, 2, 5, 5, 0, 6, 3, 7, 6, 7, 6, 5, 5,
        4, 3, 0, 3, 8, 7, 5, 0, 3, 5, 2, 6, 2, 0, 7, 7, 9, 3, 4, 0, 6, 8, 5, 8,
        9, 6, 6, 0, 4, 6, 4, 8, 6, 0, 3, 5, 3, 5, 4, 0, 8, 3, 1, 7, 2, 7, 3, 9,
        8, 0, 2, 3, 7, 3, 4, 0, 6, 2, 5, 9, 3, 7, 5, 1])
18
tensor([[[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.810

...

213
tensor([[[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.2215, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.2548, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0

217
tensor([[[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ...,  1.1559, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ...,  0.9893, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0

tensor([[[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.810

        4, 0, 0, 5, 4, 7, 7, 3, 7, 8, 0, 5, 1, 2, 9, 1])
226
tensor([[[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,

230
tensor([[[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.0438,  0.7116, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          ...,
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103]]],


        [[[-0.8103, -0.8103, -0.8103,  ..., -0.8103, -0.8103, -0.8103],
          [-0.8103, -0

In [24]:
for loader in enumerate(train_loader):
    batch_idx = loader[0]
    data = loader[1][0]
    target = loader[1][1]
    target = target if len(target) > 0 else None
    if not type(data) in (tuple, list):
        data = (data,)
    if cuda:
        data = tuple(d.cuda() for d in data)
        if target is not None:
            target = target.cuda()
    print(data[0].shape)

torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([256, 1, 28, 28])
torch.Size([25

KeyboardInterrupt: 

In [23]:
for loader in enumerate(train_dataloader_siamese):
    batch_idx = loader[0]
    data = loader[1]['img']
    target = loader[1]['target']
    target = target if len(target) > 0 else None
    if not type(data) in (tuple, list):
        data = (data,)
    if cuda:
        data = tuple(d.cuda() for d in data)
        if target is not None:
            target = target.cuda()
    print(data[0].shape)

torch.Size([256, 3, 256, 256])


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7fd734acf400>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch-1.4.0-py3.6-linux-x86_64.egg/torch/utils/data/dataloader.py", line 961, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch-1.4.0-py3.6-linux-x86_64.egg/torch/utils/data/dataloader.py", line 941, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])
torch.Size([256, 3, 256, 256])


KeyboardInterrupt: 